In [1]:
from scipy.io import loadmat
from mvlearn.embed import MCCA, KMCCA
import numpy as np
from mvlearn.compose import ViewTransformer, ConcatMerger

from sklearn.model_selection import StratifiedKFold

# 使用RandomOverSampler从少数类的样本中进行随机采样来增加新的样本使各个分类均衡
from imblearn.over_sampling import RandomOverSampler

# 通过插值生成新的样本,生成靠近在KNN中被分类错误的原样本。
from imblearn.over_sampling import ADASYN

# SMOTE: 对于少数类样本a, 随机选择一个最近邻的样本b, 然后从a与b的连线上随机选取一个点c作为新的少数类样本
from imblearn.over_sampling import SMOTE

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from keras import models
from keras import layers
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
data_path = r"D:\WorkSpace\DataSets\To石延新-苹果检测光电数据\何进荣处理后的数据\HSI_Dielectric_Data.mat"
data = loadmat(data_path)

In [3]:
from sklearn import preprocessing
#  standartize
def standartizeData(X):
    #newX = np.reshape(X, (-1, X.shape[2]))
    scaler = preprocessing.MinMaxScaler().fit(X)  
    newX = scaler.transform(X)
    newX = np.reshape(newX, (X.shape[0],X.shape[1]))
    return newX

In [4]:
y = data['Label']
Xs0 = standartizeData(data["Data1"].T)
Xs1 = standartizeData(data["Data2"].T)
Xs = np.hstack((Xs0, Xs1))

In [5]:
X_resampled_smote, y_resampled_smote = SMOTE().fit_sample(Xs, y)

In [6]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
merge = ConcatMerger()
Sensitive = []
Specificity = []
Precision = []

In [8]:
for i in range(20):
    for train, test in kfold.split(X_resampled_smote, y_resampled_smote):
        
        stand_means1 = preprocessing.MinMaxScaler()
        stand_means2 = preprocessing.MinMaxScaler()
        
        # 训练集
        X_train = X_resampled_smote[train]
        Xs_0_train = stand_means1.fit_transform(X_train[:,0:256])
        Xs_1_train = stand_means2.fit_transform(X_train[:,256:])
        X_train_list = [Xs_0_train, Xs_1_train]
        X_train_transformed = merge.fit_transform(X_train_list)
        
        clf = KNeighborsClassifier(3)
#         model = models.Sequential()
#         model.add(Dense(128, input_shape=(Xs.shape[1],), kernel_regularizer=regularizers.l2(0.0001),))
#         model.add(layers.LeakyReLU(alpha=0.1))
#         model.add(Dropout(0.5))
#         model.add(Dense(64, kernel_regularizer=regularizers.l2(0.0001),))
#         model.add(layers.LeakyReLU(alpha=0.1))
#         model.add(Dropout(0.3))
# # #         model.add(Dense(256, kernel_regularizer=regularizers.l2(0.0001),))
# # #         model.add(layers.LeakyReLU(alpha=0.1))
# # #         model.add(Dropout(0.1))
#         model.add(Dense(1, activation='sigmoid'))

#         model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

#         model.fit(X_train_transformed, y_resampled_smote[train], epochs=500, batch_size=2048, verbose=0)
        clf.fit(X_train_transformed, y_resampled_smote[train])
        # 测试集
        X_test = X_resampled_smote[test]
        Xs_0_test = stand_means1.transform(X_test[:,0:256])
        Xs_1_test = stand_means2.transform(X_test[:,256:])
        X_test_list = [Xs_0_test, Xs_1_test]
        X_test_transformed = merge.fit_transform(X_test_list)
        
        
        predictions = clf.predict(X_test_transformed)
        precisions, recall, f1score, _ = metrics.precision_recall_fscore_support(y_resampled_smote[test], predictions.round())

        Sensitive.append(precisions[0])
        Specificity.append(precisions[1])
        Precision.append(recall[0])

In [9]:
print("灵敏度：{:.4f}±{:.4f}".format(np.mean(Sensitive), np.std(Sensitive)))
print("特异度：{:.4f}±{:.4f}".format(np.mean(Specificity), np.std(Specificity)))
print("精度：{:.4f}±{:.4f}".format(np.mean(Precision), np.std(Precision)))
print("灵敏度_标准差: %.4f"%np.std(Sensitive))

灵敏度：0.9742±0.0276
特异度：0.7864±0.0445
精度：0.7300±0.0705
灵敏度_标准差: 0.0276
